# ISTA 421 / INFO 521 Fall 2023, Final project - option b
# Author: Kiwoon Hong

### Instruction: Start with the notebook above

# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split, cross_val_score, KFold, LeaveOneOut

from sklearn.metrics import roc_curve, auc

# Global paths

In [ ]:
DATA_ROOT = '../data'
FIGURE = '../figures'
PATH_TO_RATING = os.path.join(DATA_ROOT, 'rating_final.csv')
PATH_TO_USER_PAYMENT = os.path.join(DATA_ROOT, 'userpayment.csv')
PATH_TO_USER_PROFILE = os.path.join(DATA_ROOT, 'userprofile.csv')

# Load data

In [ ]:
rating_final = pd.read_csv(PATH_TO_RATING)
userpayment = pd.read_csv(PATH_TO_USER_PAYMENT)
userprofile = pd.read_csv(PATH_TO_USER_PROFILE)

# Data preprocessing

In [ ]:
#merge data
userpayment = userpayment.groupby('userID')['Upayment'].agg(lambda x: 'multiple' if len(x) > 1 else x.iloc[0]).reset_index()
rating_final = rating_final.groupby('userID')['rating'].agg(lambda x: x.mean()).reset_index()
merged_data = pd.merge(userpayment, userprofile, on='userID', how='inner')
merged_data = pd.merge(merged_data, rating_final, on='userID', how='inner')

#change output columns to category values
binsr = [0, 0.5, 1, 1.5, 2, float('inf')]
labelsr = ['[0-0.5)', '[0.5-1)', '[1-1.5)', '[1.5-2)', '[2]']
merged_data['rating_level'] = pd.cut(merged_data['rating'], bins=binsr, labels=labelsr, right=False)

#change birth_year to age
merged_data['birth_year'] = 2012 - merged_data['birth_year']
merged_data.rename(columns={'birth_year': 'age'}, inplace=True)

#drop columns
cols_to_exclude = [2, 3, 14, 15, 16, 17, 19]  
merged_data = merged_data.drop(merged_data.columns[cols_to_exclude], axis=1)

#replace '?' to NaN
merged_data.replace('?', np.nan, inplace=True)

#create output column
merged_data['churn'] = merged_data['rating_level'].apply(lambda x: 'churn' if x in ['[0-0.5)', '[0.5-1)'] else 'normal')

cols_to_exclude2 = [0, 13, 14]
df = merged_data.drop(merged_data.columns[cols_to_exclude2], axis=1)
df = df.dropna()

#data encoding - categorical to numeric
unique_values1 = df['Upayment'].unique()
print(unique_values1)
unique_values2 = df['smoker'].unique()
print(unique_values2)
unique_values3 = df['drink_level'].unique()
print(unique_values3)
unique_values4 = df['dress_preference'].unique()
print(unique_values4)
unique_values5 = df['ambience'].unique()
print(unique_values5)
unique_values6 = df['transport'].unique()
print(unique_values6)
unique_values7 = df['marital_status'].unique()
print(unique_values7)
unique_values8 = df['hijos'].unique()
print(unique_values8)
unique_values9 = df['interest'].unique()
print(unique_values9)
unique_values10 = df['personality'].unique()
print(unique_values10)
unique_values11 = df['budget'].unique()
print(unique_values11)

encoderL = LabelEncoder()
df['dress_preference'] = encoderL.fit_transform(df[['dress_preference']])
df['transport'] = encoderL.fit_transform(df[['transport']])
df['marital_status'] = encoderL.fit_transform(df[['marital_status']])
df['hijos'] = encoderL.fit_transform(df[['hijos']])
df['interest'] = encoderL.fit_transform(df[['interest']])
df['personality'] = encoderL.fit_transform(df[['personality']])
df['ambience'] = encoderL.fit_transform(df[['ambience']])
encoder1 = OrdinalEncoder(categories=[['cash', 'bank_debit_cards', 'multiple']])
df['Upayment'] = encoder1.fit_transform(df[['Upayment']])
encoder2 = OrdinalEncoder(categories=[['abstemious', 'social drinker', 'casual drinker']])
df['drink_level'] = encoder2.fit_transform(df[['drink_level']])
encoder3 = OrdinalEncoder(categories=[['low', 'medium', 'high']])
df['budget'] = encoder3.fit_transform(df[['budget']])
encoder4 = OrdinalEncoder(categories=[['false', 'true']])
df['smoker'] = encoder4.fit_transform(df[['smoker']])
encoder5 = OrdinalEncoder(categories=[['churn', 'normal']])
df['churn'] = encoder5.fit_transform(df[['churn']])
df['churn'] = 1 - df['churn']


#split data into input and output
inputdf = df.loc[:, df.columns.difference(['churn'])]
outputdf = df['churn']

#split data into train, validation, test (7: 1.5: 1.5)
ip_train, ip_temp, op_train, op_temp = train_test_split(inputdf, outputdf, test_size=0.3, random_state=2023)
ip_val, ip_test, op_val, op_test = train_test_split(ip_temp, op_temp, test_size=0.5, random_state=2023)

df


# Exploratory Data Analysis

In [ ]:

plt.figure(figsize=(8, 6))
sns.histplot(data=merged_data, x='rating_level', stat='count', kde=False, color='royalblue')
plt.title('Rating Distribution')
plt.xlabel('Rating level')
plt.ylabel('Frequency')
file_path1 = os.path.join(FIGURE, 'Rating_Distribution.png')
plt.savefig(file_path1)
plt.show()

plt.figure(figsize=(8, 6))
sns.histplot(data=merged_data, x='churn', stat='count', kde=False, color='royalblue')
plt.title('Churn Distribution')
plt.xlabel('churn')
plt.ylabel('Frequency')
file_path2 = os.path.join(FIGURE, 'Churn_Distribution.png')
plt.savefig(file_path2)
plt.show()

plt.figure(figsize=(4, 3))
sns.histplot(data=merged_data, x='age', stat='count', kde=False, color='royalblue')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

# Prediction

## Logistic Regression

In [ ]:
kfold = KFold(n_splits=20, shuffle=True, random_state=2023)
loo = LeaveOneOut()

modelLR = LogisticRegression()

#train - validation - test set
modelLR.fit(ip_train, op_train)
accuracy_LR_val = modelLR.score(ip_val, op_val)
print(f'\nLR validation set accuracy: {accuracy_LR_val:.4f}')
accuracy_LR_test = modelLR.score(ip_test, op_test)
print(f'LR test set accuracy: {accuracy_LR_test:.4f}')

#K-fold-CV
scoresLR = cross_val_score(modelLR, inputdf, outputdf, cv=kfold, scoring='accuracy')
for i, score in enumerate(scoresLR, 1):
    print(f'Fold {i}: Accuracy = {score:.4f}')

#LOOCV
inputdf_array = np.array(inputdf)
outputdf_array = np.array(outputdf)
accuraciesLR = []
probsLR = []
true_labelsLR = []
for train_index, test_index in loo.split(inputdf_array):
    ip_train, ip_test = inputdf_array[train_index], inputdf_array[test_index]
    op_train, op_test = outputdf_array[train_index], outputdf_array[test_index]
    modelLR.fit(ip_train, op_train)
    accuracyLR = modelLR.score(ip_test, op_test)
    accuraciesLR.append(accuracyLR)
    op_pred_proba = modelLR.predict_proba(ip_test)[:, 1]
    probsLR.extend(op_pred_proba)
    true_labelsLR.extend(op_test)
    

mean_accuracyLR = np.mean(accuraciesLR)
print(f'Mean Accuracy: {mean_accuracyLR:.4f}')

In [ ]:
#Visualize the coefficients
coefficientsLR = modelLR.coef_[0]
feature_names = inputdf.columns

# plot of coefficients
plt.figure(figsize=(10, 6))
plt.bar(range(len(coefficientsLR)), coefficientsLR, align='center')
plt.xticks(range(len(coefficientsLR)), feature_names, rotation=90)
plt.xlabel('Feature')
plt.ylabel('Coefficient Value')
plt.title('Logistic Regression Coefficients')
file_path3 = os.path.join(FIGURE, 'LRcoe.png')
plt.savefig(file_path3)
plt.show()

# Calculate ROC curve
fprLR, tprLR, thresholds = roc_curve(true_labelsLR, probsLR)

# Calculate AUC
roc_aucLR = auc(fprLR, tprLR)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fprLR, tprLR, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_aucLR:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
file_path4 = os.path.join(FIGURE, 'LRROC.png')
plt.savefig(file_path4)
plt.show()

## SVM(linear)

In [ ]:
modelSVM = SVC(kernel='linear', probability=True)

#LOOCV
accuraciesSVM = []
probsSVM = []
true_labelsSVM = []
for train_index, test_index in loo.split(inputdf_array):
    ip_train, ip_test = inputdf_array[train_index], inputdf_array[test_index]
    op_train, op_test = outputdf_array[train_index], outputdf_array[test_index]
    modelSVM.fit(ip_train, op_train)
    accuracySVM = modelSVM.score(ip_test, op_test)
    accuraciesSVM.append(accuracySVM)
    op_pred_proba = modelSVM .predict_proba(ip_test)[:, 1]
    probsSVM.extend(op_pred_proba)
    true_labelsSVM.extend(op_test)
mean_accuracySVM = np.mean(accuraciesSVM)
print(f'Mean Accuracy: {mean_accuracySVM:.4f}')

In [ ]:
#Visualize the coefficients
coefficientsSVM = modelSVM.coef_.ravel()
feature_names = inputdf.columns

# Plot
plt.figure(figsize=(10, 6))
plt.bar(range(len(coefficientsSVM)), coefficientsSVM, align='center')
plt.xticks(range(len(coefficientsSVM)), feature_names, rotation=90)
plt.xlabel('Feature')
plt.ylabel('Coefficient Value')
plt.title('SVM Coefficients')
file_path5 = os.path.join(FIGURE, 'SVMcoe.png')
plt.savefig(file_path5)
plt.show()

# Calculate ROC curve
fprSVM, tprSVM, thresholds = roc_curve(true_labelsSVM, probsSVM)

# Calculate AUC
roc_aucSVM = auc(fprSVM, tprSVM)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fprSVM, tprSVM, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_aucSVM:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
file_path6 = os.path.join(FIGURE, 'SVMROC.png')
plt.savefig(file_path6)
plt.show()

## Random Forest

In [ ]:
modelRF = RandomForestClassifier(n_estimators=100, random_state=2023)

#LOOCV
accuraciesRF = []
probsRF = []
true_labelsRF = []
for train_index, test_index in loo.split(inputdf_array):
    ip_train, ip_test = inputdf_array[train_index], inputdf_array[test_index]
    op_train, op_test = outputdf_array[train_index], outputdf_array[test_index]
    modelRF.fit(ip_train, op_train)
    accuracyRF = modelRF.score(ip_test, op_test)
    accuraciesRF.append(accuracyRF)
    op_pred_proba = modelRF.predict_proba(ip_test)[:, 1]
    probsRF.extend(op_pred_proba)
    true_labelsRF.extend(op_test)
mean_accuracyRF = np.mean(accuraciesRF)
print(f'Mean Accuracy: {mean_accuracyRF:.4f}')

In [ ]:
#visualize the feature importances
feature_importances = modelRF.feature_importances_
feature_names = inputdf.columns
indices = np.argsort(feature_importances)[::-1]

# plot
plt.figure(figsize=(5, 3))
plt.bar(range(len(feature_importances)), feature_importances[indices], align='center')
plt.xticks(range(len(feature_importances)), feature_names[indices], rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Random Forest Feature Importances')
file_path7 = os.path.join(FIGURE, 'RFfi')
plt.savefig(file_path7)
plt.show()

# Calculate ROC curve
fprRF, tprRF, thresholds = roc_curve(true_labelsRF, probsRF)

# Calculate AUC
roc_aucRF = auc(fprRF, tprRF)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fprRF, tprRF, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_aucRF:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
file_path8 = os.path.join(FIGURE, 'RFROC.png')
plt.savefig(file_path8)
plt.show()

## Naive Bayes

In [ ]:
modelNB = GaussianNB()

#LOOCV
accuraciesNB = []
probsNB = []
true_labelsNB = []
for train_index, test_index in loo.split(inputdf_array):
    ip_train, ip_test = inputdf_array[train_index], inputdf_array[test_index]
    op_train, op_test = outputdf_array[train_index], outputdf_array[test_index]
    modelNB.fit(ip_train, op_train)
    accuracyNB = modelNB.score(ip_test, op_test)
    accuraciesNB.append(accuracyNB)
    op_pred_proba = modelNB.predict_proba(ip_test)[:, 1]
    probsNB.extend(op_pred_proba)
    true_labelsNB.extend(op_test)
    
mean_accuracyNB = np.mean(accuraciesNB)
print(f'Mean Accuracy: {mean_accuracyNB:.4f}')

In [ ]:
# Calculate ROC curve
fprNB, tprNB, thresholds = roc_curve(true_labelsNB, probsNB)

# Calculate AUC
roc_aucNB = auc(fprNB, tprNB)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fprNB, tprNB, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_aucNB:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
file_path9 = os.path.join(FIGURE, 'NBROC')
plt.savefig(file_path9)
plt.show()